In [2]:
import pandas as pd

quick = True

if quick:
    start_year, end_year, chk, sampled, filename = [2005, 2021, False, True, 'df-light.pkl']
else:
    start_year, end_year, chk, sampled, filename = [2005, 2021, True, False, 'df-full.pkl']

from sklearn.model_selection import train_test_split

df = pd.read_pickle(f'./{filename}')
data = df.iloc[:, 1:]
target = df['grav']

for col in data.columns:
    data[col] = data[col].astype('object')
print(data.info())

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=222)
# X_train, y_train, X_test, y_test = pt.train_test_split_along_time(data, target, 2018)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99800 entries, 812244 to 2428776
Data columns (total 28 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   place    99800 non-null  object
 1   catu     99800 non-null  object
 2   sexe     99800 non-null  object
 3   trajet   99800 non-null  object
 4   locp     99800 non-null  object
 5   actp     99800 non-null  object
 6   etatp    99800 non-null  object
 7   an       99800 non-null  object
 8   mois     99800 non-null  object
 9   lum      99800 non-null  object
 10  agg      99800 non-null  object
 11  int      99800 non-null  object
 12  atm      99800 non-null  object
 13  col      99800 non-null  object
 14  dep      99800 non-null  object
 15  catr     99800 non-null  object
 16  circ     99800 non-null  object
 17  nbv      99800 non-null  object
 18  vosp     99800 non-null  object
 19  prof     99800 non-null  object
 20  plan     99800 non-null  object
 21  surf     99800 non-null  obj

In [ ]:
import time
from my_libs.encoder_custom import EncoderCustom

start_time = time.time()

# cols_target_encoded = ['dep', 'age']
cols_target_encoded = []
cols_onehot_encoded = X_train.columns.drop(cols_target_encoded)

encoder = EncoderCustom(cols_target_encoded=cols_target_encoded, cols_onehot_encoded=cols_onehot_encoded)

X_train_enc, y_train_enc = encoder.transform(X_train, y_train, 'Train')
X_test_enc,  y_test_enc  = encoder.transform(X_test,  y_test,  'Test')

print("--- Features encoding performed in %s seconds ---" % (time.time() - start_time))

In [ ]:
from my_libs.model_evaluator import ModelEvaluator

params_dt = {'max_depth': 15, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_split': 3}
model_type = 'DecisionTreeClassifier'

evaluator = ModelEvaluator(model_type=model_type, params=params_dt, X_train=X_train_enc, y_train=y_train_enc, X_test=X_test_enc, y_test=y_test_enc)
evaluator.evaluate()

In [ ]:
from my_libs.model_evaluator import ModelEvaluator

# params_rf_old = {'n_estimators': 34, 'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 2}
# params_rf = {'n_estimators': 57, 'criterion': 'entropy', 'max_depth': 39, 'min_samples_split': 4}
params_rf = {'criterion': 'gini', 'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 50}
model_type = 'RandomForestClassifier'

evaluator = ModelEvaluator(model_type=model_type, params=params_rf, X_train=X_train_enc, y_train=y_train_enc, X_test=X_test_enc, y_test=y_test_enc)
evaluator.evaluate()

In [ ]:
from my_libs import encoder_custom
from sklearn.ensemble import RandomForestClassifier
from shapash import SmartExplainer

model = RandomForestClassifier(criterion='gini', max_depth=20, min_samples_split=2, n_estimators=50)
model.fit(X_train_enc, y_train_enc)

# cols_target_encoded = []
# cols_onehot_encoded = X_train.columns.drop(cols_target_encoded)
# encoder = EncoderCustom(cols_target_encoded=cols_target_encoded, cols_onehot_encoded=cols_onehot_encoded)

xpl = SmartExplainer(
    model=model,
    # features_dict=house_dict,  # Optional parameter
    # preprocessing=encoder, # Optional: compile step can use inverse_transform method
    # postprocessing=postprocess # Optional: see tutorial postprocessing
)

tmp_df = X_test_enc
tmp_df['target'] = y_test_enc.values

# xpl.compile(x=X_test_enc, y_target=y_test_enc)
xpl.compile(x=tmp_df.drop('target',1), y_target=tmp_df.target)



In [28]:
tmp_df['target'] = y_test_enc.values
tmp_df.target

0        0
1        0
2        0
3        0
4        0
        ..
19955    0
19956    0
19957    0
19958    1
19959    0
Name: target, Length: 19960, dtype: int32